<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/rest_api_topics_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok


In [2]:
!git clone https://github.com/NikolasGialitsis/LDA2vec

fatal: destination path 'LDA2vec' already exists and is not an empty directory.


In [3]:
# -*- coding: utf-8 -*-
"""LDA2vec_inference.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1szh4AuxOleuFERJPzlqehK-yPRVwka2P
"""


!pip install spacy
!pip install jellyfish
!pip install -r /content/requirements.txt
!pip install pylda2vec




In [0]:
!cd LDA2vec/
!python -m spacy download en

In [32]:

# Commented out IPython magic to ensure Python compatibility.
%cd /content/LDA2vec/lda2vec
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import json



def getTopicComposition(data):
  #with open('/content/LDA2vec/new_document.json') as f:
  titles = []
  abstracts = []
  concats = []
  concat = data.split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  print('title:',title)
  print('abstract',abstract[:100])
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)

  stop_words = set(stopwords.words('english')) 
  new_text = []
  for text in concats.copy():
    word_tokens = word_tokenize(text) 
    #remove stopwords
    text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
    #remove punctuation
    #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

    text = ''.join([w for w in text if w not in string.punctuation])
    new_text.append(text) 

  max_length = 10000
  tokens , vocab = preprocess.tokenize(new_text, max_length, merge=False,n_threads=4)

  corpus = Corpus()
  # Make a ranked list of rare vs frequent words
  corpus.update_word_count(tokens)
  corpus.finalize()
  # The tokenization uses spaCy indices, and so may have gaps
  # between indices for words that aren't present in our dataset.
  # This builds a new compact index
  compact = corpus.to_compact(tokens)
  # Remove extremely rare words
  pruned = corpus.filter_count(compact, min_count=0)
  # Convert the compactified arrays into bag of words arrays
  bow = corpus.compact_to_bow(pruned)
  # Words tend to have power law frequency, so selectively
  # downsample the most prevalent words
  clean = corpus.subsample_frequent(pruned)
  # Now flatten a 2D array of document per row and word position
  # per column to a 1D array of words. This will also remove skips
  # and OoV words
  doc_ids = np.arange(pruned.shape[0])
  flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

  # Commented out IPython magic to ensure Python compatibility.
  # %cd /content

  fn_weights = '/content/LDA2vec/weights.pkl'
  fn_sampler = '/content/LDA2vec/sampler.pkl'
  fn_factors = '/content/LDA2vec/factors.pkl'
  weights = pickle.load(open(fn_weights, 'rb'))
  sampler = pickle.load(open(fn_sampler, 'rb'))
  factors = pickle.load(open(fn_factors, 'rb'))
  #vocab = pickle.load(open('vocab.pkl', 'rb'))

  fn_corpus = '/content/LDA2vec/corpus.pkl'
  fn_flatnd = '/content/LDA2vec/flattened.npy'
  fn_docids = '/content/LDA2vec/doc_ids.npy'

  # Model Parameters
  # Number of documents
  n_docs = doc_ids.max() + 1
  # Number of unique words in the vocabulary
  n_vocab = flattened.max() + 1
  # 'Strength' of the dircihlet prior; 200.0 seems to work well
  clambda = 200.0
  # Number of topics to fit
  n_topics = int(os.getenv('n_topics', 10))
  batchsize = 16
  # Power for neg sampling
  power = float(os.getenv('power', 0.75))
  # Intialize with pretrained word vectors
  pretrained = bool(int(os.getenv('pretrained', True)))
  # Sampling temperature
  temperature = float(os.getenv('temperature', 1.0))
  # Number of dimensions in a single word vector
  n_units = int(os.getenv('n_units', 30))
  # Get the string representation for every compact key
  words = corpus.word_list(vocab)[:n_vocab]
  print(words)
  # How many tokens are in each document
  doc_idx, lengths = np.unique(doc_ids, return_counts=True)
  doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
  doc_lengths[doc_idx] = lengths

  # Count all token frequencies
  tok_idx, freq = np.unique(flattened, return_counts=True)
  term_frequency = np.zeros(n_vocab, dtype='int32')
  term_frequency[tok_idx] = freq

  model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                  n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                  n_samples=15, power=power, temperature=temperature)
  #vectors = np.load('vectors.npy')
  #model.sampler.W.data[:, :] =  vectors[:n_vocab, :]

  #vocab = pickle.load(open(fn_vocab, 'rb'))
  corpus = pickle.load(open(fn_corpus, 'rb'))
  old_flattened = np.load(fn_flatnd)
  n_vocab = old_flattened.max() + 1
  words = corpus.word_list(vocab)[:n_vocab]



  #model.sampler.W.data[:, :] = new_text[:n_vocab, :]

  data = prepare_topics(cuda.to_cpu(weights.copy()),
                        cuda.to_cpu(factors.copy()),
                        cuda.to_cpu(sampler.copy()),
                        words)
  #add new randomized vector for new doc

  random_doc_topic = np.random.dirichlet(np.ones(10),size=1)
  first_doc = data['doc_topic_dists'][0]

  num_docs = data['doc_topic_dists'].shape[0]
  data['doc_topic_dists'] = np.append(arr=data['doc_topic_dists'],values=random_doc_topic)
  data['doc_topic_dists'] = data['doc_topic_dists'].reshape(num_docs+1,n_topics)


  print(data['doc_topic_dists'].shape)
  print('random_doc = ',random_doc_topic)

  model.to_gpu()
  optimizer = O.Adam()
  optimizer.setup(model)
  clip = chainer.optimizer.GradientClipping(5.0)
  optimizer.add_hook(clip)
  j = 0
  epoch = 0
  fraction = batchsize * 1.0 / flattened.shape[0]
  progress = shelve.open('progress.shelve')

  n_epochs = 1500

  for epoch in range(1):
      data = prepare_topics(cuda.to_cpu(weights.copy()),
                            cuda.to_cpu(factors.copy()),
                            cuda.to_cpu(sampler.copy()),
                            words)
      #top_words = print_top_words_per_topic(data)
      data['doc_lengths'] = doc_lengths
      data['term_frequency'] = term_frequency
      np.savez('topics.pyldavis', **data)
      #print(data)
      for d, f in utils.chunks(batchsize, np.array(doc_ids), flattened):
          t0 = time.time()
          model.cleargrads()
          #optimizer.use_cleargrads(use=False)
          l = model.fit_partial(d.copy(), f.copy(),update_only_docs=True)
          print(l)
          print("after partial fitting:", l)
          prior = model.prior()
          loss = prior * fraction
          loss.backward()
          optimizer.update()
          msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
                "P:{prior:1.3e} R:{rate:1.3e}")
          prior.to_cpu()
          loss.to_cpu()
          t1 = time.time()
          dt = t1 - t0
          rate = batchsize / dt
          logs = dict(loss=float(l), epoch=epoch, j=j,
                      prior=float(prior.data), rate=rate)
          print(msg.format(**logs))
          j += 1

  print('==== MAKE SURE THAT THE OTHER DOCS DONT CHANGE')
  print(first_doc)
  print(data['doc_topic_dists'][0])
  print('\n==================================')
  print('==== MAKE SURE THAT THE NEW DOC HAS CHANGED')
  print(random_doc_topic)
  print(data['doc_topic_dists'][-1])

  t=0
  print('New Document Composition\n')
  for d in data['doc_topic_dists'][-1]:
    print('\t',round(d*100,2),'\t percent of Topic',t)
    t+=1
  return data['doc_topic_dists'][-1]


/content/LDA2vec/lda2vec
Using GPU:0
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import Flask, jsonify
from flask import abort
from flask import make_response
from flask import request

app = Flask(__name__)
run_with_ngrok(app)
tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]


@app.route("/")
def home():
    return "<h1>Infer Topic Compositions on Google Colab!</h1>"

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    print(task['title'])
    composition = getTopicComposition(task['title'])
    print(composition) 
    return jsonify({'topic composition': str(composition)}), 201

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://64ce50df.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
A covid19 paper ------------------------------ Covid19 is a disease caused by the coronavirus SARS-COV2 and scientists from the united states have tried to develop vaccine
title: A covid19 paper 
abstract  Covid19 is a disease caused by the coronavirus SARS-COV2 and scientists from the united states have


INFO:werkzeug:127.0.0.1 - - [27/Apr/2020 12:03:28] "POST /todo/api/v1.0/tasks HTTP/1.1" 201 -



['out_of_vocabulary', 'out_of_vocabulary', '<SKIP>', 'covid19', 'vaccine', 'develop', 'tried', 'states', 'united', 'scientists', 'sarscov2', 'coronavirus', 'caused', 'disease', 'paper']
(745, 10)
random_doc =  [[0.04585225 0.27356406 0.09677201 0.09151243 0.10756527 0.19214536
  0.04543014 0.01054596 0.11823917 0.01837336]]
359.89133
after partial fitting: 359.89133
J:00000 E:00000 L:3.599e+02 P:-2.100e+01 R:3.500e+02
==== MAKE SURE THAT THE OTHER DOCS DONT CHANGE
[0.10384709 0.08276632 0.10338711 0.07900679 0.1079841  0.11230687
 0.09587169 0.10159854 0.1013382  0.11189319]
[0.10384709 0.08276632 0.10338711 0.07900679 0.1079841  0.11230687
 0.09587169 0.10159854 0.1013382  0.11189319]

==== MAKE SURE THAT THE NEW DOC HAS CHANGED
[[0.04585225 0.27356406 0.09677201 0.09151243 0.10756527 0.19214536
  0.04543014 0.01054596 0.11823917 0.01837336]]
[0.09601615 0.09627663 0.10512727 0.09975246 0.11829065 0.09382185
 0.09439588 0.10970131 0.08970814 0.09690975]
New Document Composition

	 9.